In [11]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langsmith import Client

In [12]:
client = Client()

In [3]:
model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.05,
    timeout=None,
    max_retries=2,
)

In [13]:
template = """
You are an assistant for improving CVs for a given job description.
Use the following job description and CV to suggest improvements.
Do not make up any information, only use the information provided.
Try not to overuse words like "spearheaded", "expert", "innovative", but do not be afraid to suggest changes.
Quantify where possible, but do not make up numbers.
If you think key skills are missing, suggest what they should be.
If you think the CV is missing information, suggest what it should be.
If you think the CV is not written in a professional tone, suggest how it should be improved.
Keep the answer concise and in British English.

{job_description}

{cv}

Suggested improvements to the CV:
"""

review_prompt = PromptTemplate.from_template(template)
review_chain = review_prompt | model

client.push_prompt("cv-reviewer", object=review_chain)

'https://smith.langchain.com/prompts/cv-reviewer/b287ad53?organizationId=4791d9fe-98f1-47bb-b116-297cd74a3dc0'

In [14]:
# Step 2: Rewrite sections of the CV based on the suggestions
rewrite_template = """
Given the following CV and the suggested improvements, rewrite the relevant sections of the CV incorporating the suggestions.
Only rewrite the sections that need improvement based on the suggestions. Keep the rest of the CV unchanged.
If you are unsure if the person has the skill, leave it unchanged.
Do not make up information, only use the information provided.
If you are unsure of what to rewrite, leave it unchanged.

CV:
{cv}

Suggested improvements:
{suggestions}

Rewritten CV sections:
"""

rewrite_prompt = PromptTemplate.from_template(rewrite_template)
rewrite_chain = rewrite_prompt | model
client.push_prompt("cv-rewrite", object=rewrite_chain)

'https://smith.langchain.com/prompts/cv-rewrite/5981f097?organizationId=4791d9fe-98f1-47bb-b116-297cd74a3dc0'

In [15]:
review_chain = client.pull_prompt("cv-reviewer", include_model=True)
rewrite_chain = client.pull_prompt("cv-rewrite", include_model=True)

In [25]:
review_chain

PromptTemplate(input_variables=['cv', 'job_description'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'cv-reviewer', 'lc_hub_commit_hash': 'b287ad531919c83d966bb025884ead8839fcdb711837efb6ccea4e711bd3ef47'}, template='\nYou are an assistant for improving CVs for a given job description.\nUse the following job description and CV to suggest improvements.\nDo not make up any information, only use the information provided.\nTry not to overuse words like "spearheaded", "expert", "innovative", but do not be afraid to suggest changes.\nQuantify where possible, but do not make up numbers.\nIf you think key skills are missing, suggest what they should be.\nIf you think the CV is missing information, suggest what it should be.\nIf you think the CV is not written in a professional tone, suggest how it should be improved.\nKeep the answer concise and in British English.\n\n{job_description}\n\n{cv}\n\nSuggested improvements to the CV:\n')
| ChatAnthropic(mod

In [29]:
def run(job_description: str, cv: str) -> tuple[str, str]:
    # Remove the tools and agent initialization since no tools are being used
    review_result = review_chain.invoke({
        "job_description": job_description,
        "cv": cv
    })
    suggestions = review_result.content

    print("Suggested improvements to the CV:")
    print(suggestions)

    rewrite_result = rewrite_chain.invoke({
        "cv": cv,
        "suggestions": suggestions
    })
    rewritten_sections = rewrite_result.content

    print("\nRewritten CV sections:")
    print(rewritten_sections)

    return suggestions, rewritten_sections

In [22]:
with open("tmp/cv.txt", "r") as file:
    cv = file.read()

In [23]:
with open("tmp/meta.txt", "r") as file:
    job_description = file.read()

In [30]:
r = run(job_description, cv)

Suggested improvements to the CV:
Based on the Meta job description, here are the key suggested improvements for the CV:

1. Technical Leadership Emphasis
- Add specific examples of team leadership and mentoring from Clarity AI role
- Quantify the size of teams led/influenced
- Highlight any experience setting technical direction and ML best practices

2. Production/Scale Focus
- Further emphasize experience with large-scale ML systems
- Add details about system reliability and performance metrics
- Include more information about debugging and troubleshooting production issues

3. Missing Technical Skills
- Add experience with C/C++ and Java if applicable
- Mention any experience with TensorFlow
- Include any recommendation systems experience
- Highlight any work with distributed computing/GPU optimization

4. Project Impact
- Add more business impact metrics to existing achievements
- Include examples of cross-team collaboration on large initiatives
- Demonstrate experience with full 